In [5]:
pip install pytesseract

In [8]:
import pandas as pd
import os
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
import pytesseract
from PIL import Image

# Load dataset
file_path = "/content/skindataall.csv"
data = pd.read_csv(file_path)

In [ ]:
# Preprocessing
# ----------------------------
# Extract relevant columns for products
product_features = ['Product_id', 'Ingredients_Cleaned', 'Skin_Type', 'Skin_Tone', 'Price', 'Category']
product_data = data[product_features]

In [6]:

# ----------------------------
# Dynamic NLP-Based Content Filtering
# ----------------------------
# Use a pre-trained SentenceTransformer for ingredient embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for product ingredients
ingredient_embeddings = embedding_model.encode(product_data['Ingredients_Cleaned'], convert_to_tensor=True)

# Compute cosine similarity between product embeddings
content_similarity = cosine_similarity(ingredient_embeddings.cpu().numpy())

# Define a list of harmful ingredients
harmful_ingredients = [
    "paraben", "sulfate", "formaldehyde", "phthalates", "triclosan", "oxybenzone"
]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Function to detect harmful ingredients in a product
def detect_harmful_ingredients(ingredients):
    # Normalize by converting to lowercase and removing extra spaces
    ingredients = ingredients.lower().strip()
    harmful_found = [ingredient for ingredient in harmful_ingredients if ingredient.lower() in ingredients]
    return harmful_found


In [10]:
# Function to recommend products based on user inputs
def recommend_products(user_skin_type, user_skin_tone, price_range, category, top_n=5):
    # Filter products based on skin type, skin tone, price range, and category
    filtered_products = product_data[
        (product_data['Skin_Type'].str.contains(user_skin_type, case=False, na=False)) &
        (product_data['Skin_Tone'].str.contains(user_skin_tone, case=False, na=False)) &
        (product_data['Price'] >= price_range[0]) &
        (product_data['Price'] <= price_range[1]) &
        (product_data['Category'].str.contains(category, case=False, na=False))
    ]

    recommendations = []
    for idx, row in filtered_products.iterrows():
        harmful_found = detect_harmful_ingredients(row['Ingredients_Cleaned'])
        if len(harmful_found) == 0:  # Only recommend products that don't have harmful ingredients
            recommendations.append(
                {"Product_id": row['Product_id'], "Price": row['Price']}  # Hide category and harmful ingredients
        )
        if len(recommendations) >= top_n:
            break

    if len(recommendations) == 0:
        # If no products match, suggest an alternative outside the price range
        print("\nSorry for the inconvenience. Unfortunately, we couldn't find any products within your price range. However, we recommend the following product:")
        fallback_product = product_data.iloc[0]  # For example, picking the first product in the dataset
        print(f"Product ID: {fallback_product['Product_id']}, Price: {fallback_product['Price']}")

    return recommendations

In [11]:
# ----------------------------
# User Interaction
# ----------------------------

# Function to add new product data to the CSV file
def add_new_product():
    # Get user input for new product data
    username = input("Enter your username: ")
    skin_tone = input("Enter your skin tone (e.g., fair, medium, olive, tan, deep): ").strip().lower()
    skin_type = input("Enter your skin type (e.g., dry, normal, oily, sensitive): ").strip().lower()
    eye_color = input("Enter your eye color: ").strip().lower()
    hair_color = input("Enter your hair color: ").strip().lower()
    rating_star = float(input("Enter the rating star (1-5): "))
    review = input("Enter your review: ")
    product = input("Enter the product name: ")
    brand = input("Enter the brand: ")
    price = float(input("Enter the product price: "))
    rating = float(input("Enter the rating: "))
    ingredients = input("Enter the product ingredients (comma-separated): ")
    combination = input("Enter if it suits combination skin (yes/no): ").strip().lower()
    dry = input("Enter if it suits dry skin (yes/no): ").strip().lower()
    normal = input("Enter if it suits normal skin (yes/no): ").strip().lower()
    oily = input("Enter if it suits oily skin (yes/no): ").strip().lower()
    sensitive = input("Enter if it suits sensitive skin (yes/no): ").strip().lower()
    category = input("Enter the product category (e.g., moisturizer, cleanser, serum): ").strip().lower()
    product_url = input("Enter the product URL: ").strip()
    user_id = input("Enter your user ID: ").strip()
    product_id = input("Enter the product ID: ").strip()
    review_cleaned = input("Enter cleaned review text: ")
    good_stuff = input("Enter the good stuff in the product: ")

    # Create a new row of data
    new_data = {
        "Username": username,
        "Skin_Tone": skin_tone,
        "Skin_Type": skin_type,
        "Eye_Color": eye_color,
        "Hair_Color": hair_color,
        "Rating_Star": rating_star,
        "Review": review,
        "Product": product,
        "Brand": brand,
        "Price": price,
        "Rating": rating,
        "Ingredients": ingredients,
        "Combination": combination,
        "Dry": dry,
        "Normal": normal,
        "Oily": oily,
        "Sensitive": sensitive,
        "Category": category,
        "Product_Url": product_url,
        "User_id": user_id,
        "Product_id": product_id,
        "Review_Cleaned": review_cleaned,
        "Good_Stuff": good_stuff,
        "Ing_Tfidf": ""  # You can add TF-IDF vectorization if needed
    }

    # Check if the file exists and has headers
    if os.path.exists(file_path):
        # Read the existing CSV to make sure headers are present
        existing_data = pd.read_csv(file_path)
        # Append the new row if the headers already exist
        new_row_df = pd.DataFrame([new_data])
        new_row_df.to_csv(file_path, mode='a', header=False, index=False)
    else:
        # If file doesn't exist, create it and add headers
        new_row_df = pd.DataFrame([new_data])
        new_row_df.to_csv(file_path, mode='w', header=True, index=False)

    print("New product data added successfully!")

In [12]:
# Function to check harmful ingredients from an image
def check_harmful_ingredients_from_image(image_path):
    try:
        # Load the image using PIL
        img = Image.open(image_path)
        # Use pytesseract to extract text from the image
        extracted_text = pytesseract.image_to_string(img)
        print("Extracted Text from Image:", extracted_text)

        # Check for harmful ingredients in the extracted text
        harmful_found = detect_harmful_ingredients(extracted_text)

        if harmful_found:
            print(f"Harmful ingredients found: {', '.join(harmful_found)}")
        else:
            print("No harmful ingredients found.")

    except Exception as e:
        print("Error processing image:", e)

In [13]:
def main():
    while True:
        print("\nSelect an option:")
        print("1. Check for harmful ingredients in a product (by entering ingredients)")
        print("2. Get product recommendations based on your skin attributes")
        print("3. Add a new product to the CSV")
        print("4. Check harmful ingredients from an image")
        print("5. Exit")
        choice = input("Enter your choice (1/2/3/4/5): ")

        if choice == '1':
            ingredients = input("Enter the product's ingredients (comma-separated): ")
            harmful_found = detect_harmful_ingredients(ingredients)
            if harmful_found:
                print(f"Harmful ingredients found: {', '.join(harmful_found)}")
            else:
                print("No harmful ingredients found.")

        elif choice == '2':
            user_skin_type = input("Enter your skin type (e.g., dry, normal, oily, sensitive): ").strip().lower()
            user_skin_tone = input("Enter your skin tone (e.g., fair, medium, olive, tan, deep): ").strip().lower()
            price_min = float(input("Enter your minimum price: ").strip())
            price_max = float(input("Enter your maximum price: ").strip())
            category = input("Enter the product category (e.g., moisturizer, cleanser, serum): ").strip().lower()
            price_range = (price_min, price_max)

            recommendations = recommend_products(user_skin_type, user_skin_tone, price_range, category)
            if recommendations:
                print("Recommended products for you:")
                for rec in recommendations:
                    print(f"Product ID: {rec['Product_id']}, Price: {rec['Price']}")
            else:
                print("No products found matching your criteria.")

        elif choice == '3':
            add_new_product()

        elif choice == '4':
            image_path = input("Enter the path to the product image: ").strip()
            check_harmful_ingredients_from_image(image_path)

        elif choice == '5':
            print("Exiting the program.")
            break

        else:
            print("Invalid choice. Please enter 1, 2, 3, 4, or 5.")

if __name__ == "__main__":
    main()



Select an option:
1. Check for harmful ingredients in a product (by entering ingredients)
2. Get product recommendations based on your skin attributes
3. Add a new product to the CSV
4. Check harmful ingredients from an image
5. Exit
Enter your choice (1/2/3/4/5): 1
Enter the product's ingredients (comma-separated): paraben
Harmful ingredients found: paraben

Select an option:
1. Check for harmful ingredients in a product (by entering ingredients)
2. Get product recommendations based on your skin attributes
3. Add a new product to the CSV
4. Check harmful ingredients from an image
5. Exit
Enter your choice (1/2/3/4/5): 2
Enter your skin type (e.g., dry, normal, oily, sensitive): dry
Enter your skin tone (e.g., fair, medium, olive, tan, deep): fair
Enter your minimum price: 35
Enter your maximum price: 120
Enter the product category (e.g., moisturizer, cleanser, serum): moisturizer
Recommended products for you:
Product ID: 272, Price: 38
Product ID: 176, Price: 39
Product ID: 304, Pric